In [1]:
import numpy as np
import vedo
vedo.settings.default_backend= 'k3d'

from brainrender import Scene
import pathlib


In [2]:
from myterial import orange
from rich import print

from brainrender import Scene, settings
from brainrender.atlas_specific import GeneExpressionAPI
from brainrender.actors import Volume

In [11]:
mouse = 'FIAA45.6d'
lcm_directory = pathlib.Path(
   f"/camp/lab/znamenskiyp/home/shared/projects/turnerb_A1_MAPseq/{mouse}/LCM"
)
ROI_3D = np.load(lcm_directory / "ROI_3D_25.npy")
scene = Scene(atlas_name='allen_mouse_25um')

In [6]:
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
tab20 = cm.get_cmap("gist_ncar", 250)
tab20_cmap = tab20(np.linspace(0, 1, 20))
new_colors = tab20(np.linspace(0, 1, 180))

black = np.array([1 / 256, 1 / 256, 1 / 256, 1])

for i, row in enumerate(new_colors):
    if i == 0:
        new_colors[i, :] = black
    else:
        idx = np.random.randint(20)
        new_colors[i, :] = tab20_cmap[idx]
newcmp = ListedColormap(new_colors)

/tmp/ipykernel_151519/75824251.py:3: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  tab20 = cm.get_cmap("gist_ncar", 250)


In [12]:
bla = Volume(ROI_3D, min_value=1, voxel_size=25, cmap=newcmp)
act = scene.add(bla)
scene.jupyter = True
# render
scene.content
scene.render()

╭─────────────────────────────────╮
│                                 │
│   Scene actors                  │
│   ────────────                  │
│                                 │
│   - root (type: brain region)   │
│   - Volume (type: Volume)       │
│                                 │
╰─────────────────────────────────╯

Cannot run function _get_inset in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

In [13]:
from vedo import Plotter  # <- this will be used to render an embedded scene 
plt = Plotter()
plt.show(*scene.renderables)                                                                                  

/camp/home/turnerb/.conda/envs/MAPseq_processing_py39/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

In [ ]:
#this doesn't work - even if you run not off jupyter notebook
vedo.settings.default_backend= 'vtk'
from brainrender.video import VideoMaker
vm = VideoMaker(scene, str(lcm_directory/'vids'), 'vid1')
vm.make_video(elevation=1, roll=5)

In [13]:
import pandas as pd
barcodes_across_sample = pd.read_pickle(
    f"/camp/lab/znamenskiyp/home/shared/projects/turnerb_A1_MAPseq/{mouse}/Sequencing/barcode_matrix_soma_thresholded.pkl"
)
sample_vol_and_regions =pd.read_pickle(lcm_directory/'sample_vol_and_regions.pkl')
A1 = sample_vol_and_regions[sample_vol_and_regions['main']=='AUDp']['ROI Number'].to_list()
barcodes_across_sample = barcodes_across_sample[barcodes_across_sample.idxmax(axis=1).isin(A1)]

In [14]:
# remove tubes in ROI flatmap that aren't in normalised barcode path
tubes = np.arange(
    np.min(barcodes_across_sample.columns), np.max(barcodes_across_sample.columns), 1
)
tubes_not_in = [i for i in tubes if i not in barcodes_across_sample.columns.to_list()]
for x in tubes_not_in:
    ROI_3D[ROI_3D == x] = 0

In [18]:
#if you just want visual areas to look at
VIS_samples = sample_vol_and_regions[sample_vol_and_regions.main.str.contains('VIS')]['ROI Number'].to_list()
barcodes_across_sample[barcodes_across_sample[VIS_samples].ne(0).any(1)]

barcode_matrix = np.zeros(
    (len(barcodes_across_sample), max(barcodes_across_sample.columns.to_list()) + 1)
)

for column in barcodes_across_sample:
    barcode_matrix[:, column] = barcodes_across_sample[column].to_numpy()

# now set zero values to -1
barcode_matrix[:, 0] = 0


/tmp/ipykernel_60664/4106561458.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  barcodes_across_sample[barcodes_across_sample[VIS_samples].ne(0).any(1)]


In [21]:
barcode_index = np.random.randint(len(barcode_matrix))
barcode_matrix[:, 0] = -1
new_mat = np.log(
            1 + barcode_matrix[barcode_index, ROI_3D.astype(int)]
        )


In [55]:
np.where

2110

In [58]:
barcode_roi.max()

2.685752146934234

In [56]:
barcode_roi = np.empty_like(ROI_3D)
for i in barcodes_across_sample.columns:
    mask = np.isin(ROI_3D, i)
    barcode_roi[mask] = np.log10(barcode_matrix[barcode_index, i] + 1)

In [57]:
scene_single_neuron = Scene(atlas_name='allen_mouse_25um')
neuron_sing = Volume(barcode_roi, min_value=1, voxel_size=25, cmap='inferno')
actor_neuron = scene_single_neuron.add(neuron_sing)

scene_single_neuron.jupyter = True
# render
scene_single_neuron.content
scene_single_neuron.render()
plt = Plotter()
plt.show(*scene_single_neuron.renderables)    

╭─────────────────────────────────╮
│                                 │
│   Scene actors                  │
│   ────────────                  │
│                                 │
│   - root (type: brain region)   │
│   - Volume (type: Volume)       │
│                                 │
╰─────────────────────────────────╯

Cannot run function _get_inset in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

/camp/home/turnerb/.conda/envs/MAPseq_processing_py39/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

In [25]:
plt.show(*scene_single_neuron.renderables)    

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

In [23]:
vedo.settings.default_backend= 'vtk'
from brainrender.video import VideoMaker
vm = VideoMaker(scene, str(lcm_directory/'vids'), 'vid1')
vm.make_video(elevation=1, roll=5)

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

Saving video in /camp/lab/znamenskiyp/home/shared/projects/turnerb_A1_MAPseq/FIAA45.6a/LCM/vids

📽  Video file vid1 is open... 

Output()

AttributeError: 'NoneType' object has no attribute 'ComputeVisiblePropBounds'

In [19]:
popup_scene = Scene(atlas_name='allen_mouse_50um', title='popup')

popup_scene.add_brain_region('VISp')
popup_scene.jupyter = True
popup_scene.render() 
from vedo import Plotter  # <- this will be used to render an embedded scene 
plt = Plotter()
plt.show(*popup_scene.renderables)

Cannot run function _get_inset in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

/camp/home/turnerb/.conda/envs/MAPseq_processing_py39/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

In [8]:
source_origin = ("Posterior", "Inferior", "Right")
target_space = scene.atlas.space
import brainglobe_space as bg
mapped_stack = bg.map_stack_to(source_origin, target_space, ROI_3D)

In [29]:
scene.jupyter = True

In [11]:
from vedo import Volume
vol = Volume(mapped_stack)

In [30]:
gene_actor = geapi.griddata_to_volume(ROI_3D, min_quantile=99, cmap="inferno")

In [44]:
from brainrender.actors import Volume
bla = Volume(ROI_3D, voxel_size=25, as_surface=False, c='Reds')

In [45]:
act = scene.add(bla)

In [12]:
settings.SHOW_AXES = False

scene = Scene(inset=False)

gene = "Gpr161"
geapi = GeneExpressionAPI()
expids = geapi.get_gene_experiments(gene)
data = geapi.get_gene_data(gene, expids[1])

gene_actor = geapi.griddata_to_volume(data, min_quantile=99, cmap="inferno")
act = scene.add(gene_actor)

#ca1 = scene.add_brain_region("CA1", alpha=0.2, color="skyblue")
#ca3 = scene.add_brain_region("CA3", alpha=0.5, color="salmon")


In [39]:
from brainrender.actors import Points



def get_n_random_points_in_region(region, N):
    """
    Gets N random points inside (or on the surface) of a mes
    """

    region_bounds = region.mesh.bounds()
    X = np.random.randint(region_bounds[0], region_bounds[1], size=10000)
    Y = np.random.randint(region_bounds[2], region_bounds[3], size=10000)
    Z = np.random.randint(region_bounds[4], region_bounds[5], size=10000)
    pts = [[x, y, z] for x, y, z in zip(X, Y, Z)]

    ipts = region.mesh.inside_points(pts).coordinates
    return np.vstack(random.choices(ipts, k=N))

In [41]:
import random
scene = Scene(title="Labelled cells")

# Get a numpy array with (fake) coordinates of some labelled cells
mos = scene.add_brain_region("MOs", alpha=0.15)
coordinates = get_n_random_points_in_region(mos, 2000)

# Add to scene
scene.add(Points(coordinates, name="CELLS", colors="steelblue"))
scene.jupyter = True
# render
scene.content
scene.render()

╭─────────────────────────────────╮
│                                 │
│   Scene actors                  │
│   ────────────                  │
│                                 │
│   - root (type: brain region)   │
│   - title (type: title)         │
│   - MOs (type: brain region)    │
│   - CELLS (type: Points)        │
│                                 │
╰─────────────────────────────────╯

Cannot run function _get_inset in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

In [43]:
coordinates


array([[5192., 1040., 6347.],
       [5066.,  982., 4884.],
       [2931., 2385., 3051.],
       ...,
       [2786., 3352., 3516.],
       [4043., 2764., 7258.],
       [2792., 1902., 7110.]], dtype=float32)

In [4]:
#scene.jupyter = True
scene.add_silhouette(act)

Cannot run function add_silhouette in a jupyter notebook Try setting the correct backend before creating your 
scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

In [46]:
scene.jupyter = True

In [1]:

scene.render()

NameError: name 'scene' is not defined

In [48]:
from vedo import show                                                                                              
vedo.show(*scene.renderables)

/camp/home/turnerb/.conda/envs/MAPseq_processing_py39/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


In [ ]:
popup_scene = Scene(atlas_name='allen_mouse_50um', title='popup')

popup_scene.add_brain_region('VISp')
# Make sure it gets embedded in the window
popup_scene.jupyter = True
popup_scene.render()  # press 'Esc' to close
from vedo import Plotter  # <- this will be used to render an embedded scene 
plt = Plotter()
plt.show(*popup_scene.renderables)

: 

In [6]:
mouse = 'FIAA45.6a'
lcm_directory = pathlib.Path(
   f"/camp/lab/znamenskiyp/home/shared/projects/turnerb_A1_MAPseq/{mouse}/LCM"
)
ROI_3D = np.load(lcm_directory / "ROI_3D_25.npy")

In [8]:
from brainrender import Scene, actors
scene = Scene(atlas_name='allen_mouse_25um')
scene.add(actors.Volume(ROI_3D))

: 

In [15]:
from brainrender.actors import Volume
mesh = Volume(ROI_3D).isosurface()
scene = Scene()
scene.add(mesh)
scene.render()  # press 'Esc' to close
from vedo import Plotter  # <- this will be used to render an embedded scene 
plt = Plotter()
plt.show(*scene.renderables)

AttributeError: module 'matplotlib' has no attribute 'colormaps'

In [20]:
scene = Scene(atlas_name='allen_mouse_25um')

In [21]:
from vedo import Volume
from brainrender import Scene

In [5]:
from matplotlib import colormaps

In [26]:
vol = Volume(ROI_3D)
mesh = vol.isosurface()


AttributeError: module 'matplotlib' has no attribute 'colormaps'